# Get MIRI programs info #

**Goal:** Create a csv file with relevant information on all observed MIRI programs

**Author:** Boris Trahin, Staff Scientist II, MIRI team

**Last updated:** November 4, 2024

***
### Import useful packages

In [1]:
from astroquery.mast.missions import MastMissions
from astroquery.mast import Observations
from astropy.table import unique, vstack, Table
from datetime import date
import pandas as pd
update = date.today()

***
### Get entire catalog of MIRI observations

<div class="alert alert-block alert-info">
Set some parameters
</div>

In [2]:
save = True # save the catalogs into csv file
add_reviewer = True # add the MIRI internal program reviewer (VERY SLOW without multiprocessing...)

<div class="alert alert-block alert-info">
Get the data from MAST. This is done for each exposure types (otherwise simultaneous observations are badly referenced)
</div>

In [3]:

good_columns = ['fileSetName', 'obs_id', 'program', 'proposal_cycle', 'date_obs', 'date_beg', 'date_end',
                'visitgrp', 'observtn', 'bkgdtarg', 'visit',
                'targtype', 'exp_type', 'tsovisit', 'expripar', 'detector',
                'opticalElements', 'filter', 'subarray', 'pps_aper', 'apername',
                'miri_channel', 'miri_band', 'miri_coronmsk', 'targ_ra', 'targ_dec',
                'nresets', 'frmdivsr', 'nframes', 'ngroups', 'nints', 'readpat', 'numdthpt',
                'patttype', 'miri_dithopfr', 'miri_numdsets',
                'tframe', 'tgroup', 'effinttm','duration', 'eng_qual',
                'hga_move', 'lamp', 'pcs_mode', 'miri_cccstate', 'exp_only', 'visitsta', 'template',
                'targdesc', 'scicat', 'targcat', 'targprop', 'targname', 'obslabel', 'title',
                'pi_name', 'category', 'access']

exposure_types = ['MIR_DARKMRS', 'MIR_DARKIMG',
                  'MIR_FLATIMAGE', 'MIR_FLATMRS', 'MIR_FLATIMAGE-EXT',
                  'MIR_MRS', 'MIR_IMAGE', 
                  'MIR_LRS-SLITLESS', 'MIR_LRS-FIXEDSLIT',
                  'MIR_4QPM','MIR_LYOT','MIR_CORONCAL', 
                  'MIR_TACQ', 'MIR_TACONFIRM']

missions = MastMissions(mission='jwst')
dfs = []

for e in exposure_types:
    print(f"Getting {e} exposures...")
    results = missions.query_criteria(select_cols=good_columns,
    exp_type = e,
    instrume='MIRI',
    limit=-1,
        )
    sub_df = results.to_pandas()
    dfs.append(sub_df)
df = pd.concat(dfs)


Getting MIR_DARKMRS exposures...


Getting MIR_DARKIMG exposures...
Getting MIR_FLATIMAGE exposures...
Getting MIR_FLATMRS exposures...
Getting MIR_FLATIMAGE-EXT exposures...
Getting MIR_MRS exposures...
Getting MIR_IMAGE exposures...
Getting MIR_LRS-SLITLESS exposures...
Getting MIR_LRS-FIXEDSLIT exposures...
Getting MIR_4QPM exposures...
Getting MIR_LYOT exposures...
Getting MIR_CORONCAL exposures...
Getting MIR_TACQ exposures...
Getting MIR_TACONFIRM exposures...


<div class="alert alert-block alert-info">
Uncomment the following line to save the list of programs before the changes made in the next cell
</div>

In [4]:
# df.to_csv('./MIRI_programs_MAST.csv', index=False) # save to a csv file and remove first index column

<div class="alert alert-block alert-info">
Some columns are badly reference (e.g. MRS detector), so organize it.
One can uncomment the line df2 = df2.drop_duplicates(subset=['obs_id', 'exp_type']) to remove duplicates based on obs_id and exp_type. This will keep only 1 dither position per exposure.
</div>

In [5]:
df2 = df.reindex(good_columns, axis=1)

# Some columns are badly referenced
# df2.loc[(df2['exp_type']=='MIR_MRS'), 'detector'] = 'MIRIFUSHORT, MIRIFULONG'
# df2.loc[(df2['exp_type']=='MIR_DARKMRS'), 'detector'] = 'MIRIFUSHORT, MIRIFULONG'
df2.loc[(df2['exp_type']).str.contains('MRS'), 'detector'] = 'MIRIFUSHORT, MIRIFULONG'

df2.loc[(df2['detector']=='MIRIFUSHORT, MIRIFULONG'), 'miri_channel'] = '12, 34'

df2['readpat'] = df2['readpat'].fillna('')
df2['readpat'] = df2['readpat'].astype(str)
df2.loc[(df2['tframe']<3) & (df2['frmdivsr']==1) & (df2['nresets']==1), 'readpat'] = 'FASTR1'
df2.loc[(df2['tframe']<3) & (df2['frmdivsr']==1) & (df2['nresets']==0), 'readpat'] = 'FAST'
df2.loc[(df2['tframe']<3) & (df2['frmdivsr']==4), 'readpat'] = 'FASTGRPAVG'
df2.loc[(df2['tframe']<3) & (df2['frmdivsr']==8), 'readpat'] = 'FASTGRPAVG8'
df2.loc[(df2['tframe']<3) & (df2['frmdivsr']==32), 'readpat'] = 'FASTGRPAVG32'
df2.loc[(df2['tframe']<3) & (df2['frmdivsr']==64), 'readpat'] = 'FASTGRPAVG64'

df2.loc[(df2['tframe']>3) & (df2['frmdivsr']==1) & (df2['nresets']==1), 'readpat'] = 'SLOWR1'
df2.loc[(df2['tframe']>3) & (df2['frmdivsr']==1) & (df2['nresets']==0), 'readpat'] = 'SLOW'

df2 = df2.reset_index()
df2 = df2.drop(df2[df2['fileSetName'].str.contains('-')].index)

df2 = df2.sort_values(['program', 'observtn'],axis=0, ascending=False) # latest program first
# df2 = df2.drop_duplicates(subset=['obs_id', 'exp_type']) # Uncomment this line to remove duplicates based on obs_id and exp_type. This will keep only 1 dither position per exposure.
df2 = df2.reindex(good_columns, axis=1)

<div class="alert alert-block alert-info">
Get MIRI internal reviewer and add the corresponding column in the CSV file.

Use the function get_reviewer() in mirireviewer.py to enable multiprocessing (if multipro = True).
</div>

In [6]:
if add_reviewer:
    import mirireviewer
    multipro = True # Take 2 min with multiprocessing and 10 cpu
    programs = df2['program'].unique()
    if multipro:
        from multiprocessing import Pool
        pool = Pool()
        reviewers = pool.map(mirireviewer.get_reviewer, programs)
        for program, reviewer in zip(programs, reviewers):
            df2.loc[(df2['program']==program), 'MIRI Reviewer'] = reviewer
    else:
        for program in programs:
            reviewer = mirireviewer.get_reviewer(program)
            df2.loc[(df2['program']==program), 'MIRI Reviewer'] = reviewer

6463
6811
45324706

5365
6620
6580
4490
4343
4192
6617
4489
6151
4687
5299
6761
4094
6579
4332
4525
4488
6616
4668
6138
6751
5232
4522
4093
6565
4297
6615
4487
4583
6023
6743
5204
4520
4078
4290
6564
4486
6612
5842
4578
6677
5191
4065
4516
4278
6611
6559
4484
4575
5709
5168
6669
6610
6558
4483
4256
4038
4499
5627
4572
6607
4482
5114
6659
4246
4498
4028
6556
6604
4565
4440
5578
5105
6627
4497
4244
6555
3983
6595
4436
5491
4564
4496
6624
5057
4237
6554
3964
6591
5474
4427
4493
4560
6623
5014
4225
3960
6553
5451
6582
4540
4403
4492
6622
4829
4217
3958
6550
3930
5407
4491
6621
3743
4539
4727
4201
3954
6540
3662
3523
3921
3315
3166
3730
2926
2757
2708
2521
3655
3886
3295
3514
3158
2919
3726
2754
2667
2516
3647
3271
3869
3486
3153
3722
2896
2741
2666
3257
2511
3629
3832
3477
3077
2739
2826
3712
2662
3254
3621
3823
2508
3449
3050
2820
2736
3707
2654
3794
3571
3228
2491
3445
3034
2733
2783
3786
3703
2635
3558
3226
2459
3436
2732
3010
2782
3702
3779
3224
3557
2441
2581
2731
3435
2987
2780
3696


<div class="alert alert-block alert-info">
Save CSV file
</div>

In [7]:
if save:
    with open(f"./JWST_MIRI_programs_info_updated{date.today()}.csv", 'w') as outfile:
        df2.to_csv(outfile, index=False) # save to a csv file and remove first index column